# Comet Impact
## Importing Scipp

The canonical way to import scipp is as `sc`:

In [ ]:
import scipp as sc

After having watched the 2021 movie "Don't Look Up" we may wonder:
If such a comet is destroyed/dismantled an the debris/pieces impacts the earth's atmosphere, by how much is this going to raise the air temperature?

Far from a serious analysis, we are going to use scipp for a back-of-the-envelope calculation.
Scipp's physical units will increase our confidence that we did not mess up.
Let us start with the following assumptions:

- 100% of the comet's kinetic energy is converted into thermal energy raising the earth's air temperature.
  We ignore heating of comet material, or pieces impacting the earth's surface.
- 100% of the comet debris actually hits the atmosphere, i.e., the debris has not spread out enough to partially miss.

## Physical quantities

For starters, we need to know the mass of the atmosphere.
We can compute this from the earth surface $A_\text{air}$ and the air pressure at sea level $P_\text{air}$.
In the following we will use scalar-valued (as opposed to array-valued) quantities.
We can create these with scipp using a value and a physical unit.
The earth radius is:

In [ ]:
r_earth = sc.scalar(6371., unit='km')

We can inspect the value of this variable by typing it at the end of a notebook cell:

In [ ]:
r_earth

Above the empty tuple `()` indicates that this is a 0-D variable, i.e., it has a single value.

## Unit conversions

We know the pressure at sea level $P_\text{air} \approx 1~\text{bar}$ (we could use a more precise value, but the difference is completely irrelevant given the other assumptions and approximations made in the ballpark calculation).
Since we want to work in SI units we need to convert this to $N/m^2$.
Scipp has a built-in feature for such conversions, using the `to` method with the `unit` keyword argument:

In [ ]:
P_air = sc.scalar(1.0, unit='bar').to(unit='N/m**2')
P_air

## Physical constants

To proceed, we need to compute the mass (of air) that would excert such a pressure.
For this we need the gravitational acceleration.
The `scipp.constants` module provides this:

In [ ]:
from scipp.constants import g

The constants are scalar variables, similar to those we defined earlier ourselves:

In [ ]:
g

## Computation

We assume that $g$ is roughly constant for the relevant altitude range and obtain, using $P=F/A$ and $F = mg$.
We will also need $\pi$ to compute the earth's surface area from its radius, which we can import from `scipp.constants`:

In [ ]:
from scipp.constants import pi
A_air = 4 * pi * r_earth**2
m_air = A_air * P_air/g
m_air

We note the odd scale factor in the unit of this result.
This originates from (the square of) the unit of `km` we used for the earth radius.
Scipp does not automatically move scale factors of the unit to the value.
Instead, we need to request this by converting the unit:

In [ ]:
m_air = m_air.to(unit='kg')
m_air

We may compare to [Wikipedia](https://en.wikipedia.org/wiki/Atmosphere_of_Earth), which lists  $5.15\cdot 10^{18}~\text{kg}$, i.e., our estimation is correct so far.

Next, we need to estimate the kinetic energy of the comet.
We have $E_\text{kin} = \frac{1}{2} mv^2$.
The diameter of the comet in the movie is $9~\text{km}$.
A quick internet research shows that comets may have quite a low density, even lighter than water.
The average impact speed of comets from the outer solar system is around $50~\text{km/s}$.
Let us set:

In [ ]:
rho = 0.6 * sc.Unit('g/cm^3')  # lighter than water
r = 9.0 * sc.Unit('km') / 2
v = 50.0 * sc.Unit('km/s')

Then

In [ ]:
m = rho * 4/3 * pi * r**3
E_kin = 0.5 * m * v**2
E_kin

We also lookup the specific heat of air, $1.005~\text{J/g/K}$, and obtain a warming $\Delta T$ of the earth atmosphere by

In [ ]:
c_air = 1.005 * sc.Unit('J/(g*K)')
delta_T = E_kin/(c_air * m_air)
delta_T.to(unit='K')

## Arrays

So far we have worked with 0-D variables, i.e., quantities with a single value.
Scipp's main focus is however operations with arrays of values.
We will show in the following how we can use this with our above calculation.

Before we begin, let us combine some of the computational steps done above into a reusable function:

In [ ]:
def temperature_increase(rho, r, v):
    """
    Compute temperature increase caused by impact of object
    with density rho, radius r and velocity v.
    """
    m = rho * 4 / 3 * pi * r**3
    E_kin = 0.5 * m * v**2
    delta_T = E_kin / (c_air * m_air)
    return delta_T.to(unit='K')

We could call this function multiple times with different velocities.
If you are familiar with NumPy you may recognize that we may instead pass arrays as the arguments of this function.
The values of scipp's variables work similar to NumPy arrays.
We can create a 1-D variable of velocities:

In [ ]:
v = sc.array(dims=['velocity'], values=[20., 30., 40., 50.], unit='km/s')
v

In contrast to a plain NumPy array, the scipp variable also stores a unit and a tuple of dimension labels.
We can use this array of velocities to compute multiple temperature increases at a time:

In [ ]:
temperature_increase(rho=rho, r=r, v=v)

We could also define multiple comet radii and compute:

In [ ]:
r = sc.array(dims=['radius'], values=[2.0, 4.0, 8.0], unit='km')
temp = temperature_increase(rho=rho, r=r, v=v)
temp

The function now returns a 2-D variable.
Since `r` and `v` have different dimension labels scipp automatically computes a value for each possible combination.
This is called *broadcasting*.

The dimension labels can be used, e.g., to specify which dimension to slice.
We can obtain a view for the second radius value using:

In [ ]:
temp['radius', 1]

One thing is missing from our result:
It is difficult to identify which data value belong to which value of `r` and `v`.
This can be achieved by storing them as *coordinates* of the data, using scipp's `DataArray`, which is covered in other tutorials.